# Master file

This file should combine all the other experiments

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from matplotlib.colors import to_rgb
from matplotlib.colors import LinearSegmentedColormap


# Model and Training Params

In [ ]:
# Model Params
num_hidden = 5 # number of hidden layers. The total network has additionl 2 layers: input to hidden and hidden to output
input_dim = 2
hidden_dim = 2
output_dim = 1
activation = 'tanh' #'relu' and 'tanh' are supported

# Training Params
load_file = None
cross_entropy = True #True supported with binary classification only, False needs accuracy fix
num_epochs = 300

subfolder = 'master_file'
import os
if not os.path.exists(subfolder):
    os.makedirs(subfolder)

In [ ]:
import models.training
from models.training import make_circles_uniform

# Generate training data

# Set random seed for reproducibility
seed = np.random.randint(1000)
# seed = 163
np.random.seed(seed)
torch.manual_seed(seed)

#footnnote to display on plots to make sure that plots and model/trainign params do not get confused
footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'

n_points = 2000 #number of points in the dataset

inner_radius = 0.5
outer_radius = 1
buffer = 0.2

import importlib
importlib.reload(models.training) # Reload the module

train_loader, test_loader = make_circles_uniform(output_dim = 1, n_samples = n_points, inner_radius = 0.5, outer_radius = 1.0, buffer = 0.1, cross_entropy=cross_entropy, seed = seed, filename = subfolder + '/circles_dataset')

In [ ]:
for input, label in train_loader:
    print(input[:5], label[:5])
    break

# Define model and training


In [ ]:
# to reload models.resnet module after changes without restarting the kernel
import importlib
import models.resnets
import models.training
importlib.reload(models.resnets) # Reload the module
importlib.reload(models.training) # Reload the module
from models.resnets import ResNet
from models.training import compute_accuracy, train_model, train_until_threshold, plot_loss_curve


# MLP Constant width 2 

In [ ]:
skip_param = 0
# Train models
model_base, acc_base, losses_base = train_until_threshold(ResNet,
    train_loader, test_loader,
    load_file = load_file, max_retries=5, threshold=0.95,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=skip_param, activation=activation
)

plot_loss_curve(losses_base, title=f"Base Model Loss Curve", filename = subfolder + '/ff6hidden')

In [ ]:
model_base.parameters

In [ ]:
import importlib
import plots.plots 
from plots.plots import plot_decision_boundary, plot_level_sets
importlib.reload(plots.plots) # Reload the module

X_test, y_test = next(iter(test_loader))
plot_decision_boundary(model_base, X_test, y_test, show=True, file_name= subfolder + '/ff6hiddencirc' + str(num_hidden), footnote = footnote, amount_levels= 100)
plot_level_sets(model_base, show=True, file_name= subfolder + '/ff6hiddencirc_contour' + str(num_hidden), footnote = footnote, amount_levels= 20)



# Augmented model: width 3

In [ ]:
hidden_dim = 3
num_hidden = 1

footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'

seed = 288

model_aug, acc_aug, losses_aug = train_until_threshold(ResNet,
    train_loader, test_loader,
    load_file = load_file, max_retries=5, threshold=0.95, seed=seed,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=0, activation=activation
)

plot_loss_curve(losses_aug, title=f"Augmented Model Loss Curve", filename = subfolder + '/ff6hidden_aug')

In [ ]:
X_test, y_test = next(iter(test_loader))
footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'
plot_decision_boundary(model_aug, X_test, y_test, show=True, file_name= subfolder + '/ffaugcirc' + str(num_hidden), footnote = footnote, amount_levels= 100)
plot_level_sets(model_aug, show=True, file_name= subfolder + '/ffaugcirc_contour' + str(num_hidden), footnote = footnote, amount_levels= 20)



# ResNet model

In [ ]:
skip_param = 1 # this sets model from feed forward to residual network

num_hidden = 6
hidden_dim = 2

footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'



model_res, acc_res, losses_res = train_until_threshold(ResNet,
    train_loader, test_loader,
    load_file = load_file, max_retries=5, threshold=0.95,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=skip_param, activation=activation,
)

plot_loss_curve(losses_res, title=f"ResNet Model Loss Curve", filename = subfolder + '/ff6_res')


In [ ]:
X_test, y_test = next(iter(test_loader))
plot_decision_boundary(model_res, X_test, y_test, show=True, file_name= subfolder + '/ff6resnetcirc' + str(num_hidden), footnote = footnote, amount_levels= 100)
plot_level_sets(model_res, show=True, file_name= subfolder + '/ff6resnetcirc_contour' + str(num_hidden), footnote = footnote, amount_levels= 10, plotrange=[-2.5,2.5])

In [ ]:
import importlib; import plots.plots; importlib.reload(plots.plots)
from plots.plots import plot_singular_values_of_weightmatrix, plot_EVmod_of_weightmatrix

X_test, y_test = next(iter(test_loader))
num_epochs = 100

models_with_skipparams = {}


skip_values = np.linspace(0, 5, 6)  # e.g., [0.0, 0.125, ..., 1.0]
# skip_values = [0, 0, 0 , 0 , 0]  # e.g., [0.0, 0.125, ..., 1.0]
n_cols = 3
n_rows = int(np.ceil(len(skip_values) / n_cols))
fig1, axes1 = plt.subplots(n_rows, n_cols, figsize=(3.5 * n_cols, 4 * n_rows), facecolor='white', dpi = 900)
plt.subplots_adjust(wspace=0.05, hspace=0.) 
fig2, axes2 = plt.subplots(n_rows, n_cols, figsize=(3.5 * n_cols, 4 * n_rows), facecolor='white', dpi = 900)
plt.subplots_adjust(wspace=0.05, hspace=0.) 

# num_epochs = 10
for idx, skip in enumerate(skip_values):
    print(f"Training model with skip_param = {skip:.2f}")
    
    seed = 14 #10 had complex eigenvalues
    print(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # activation = 'tanh'

    model = ResNet(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim,
                   num_hidden=num_hidden, skip_param=skip, activation=activation)

    train_model(model, train_loader, test_loader, epochs = num_epochs, early_stopping = False)
    
    models_with_skipparams[skip] = model

    ax1 = axes1.flatten()[idx]
    plot_decision_boundary(model, X_test, y_test, title=f"Skip: {skip:.2f}", ax=ax1, show=False, colorbar=False, show_points=False, amount_levels=100)
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.axis('tight')
    ax1.set_aspect('equal') 
    
    ax2 = axes2.flatten()[idx]
    plot_EVmod_of_weightmatrix(model, ax = ax2, log_scale = False, title=f"Skip: {skip:.2f}")
    
    # ax.set_xticks([])
    # ax.set_yticks([])
    # ax.set_xlabel('')
    # ax.set_ylabel('')
    # ax.axis('tight')
    # ax.set_aspect('equal') 
    

# Hide unused subplots
for i in range(len(skip_values), len(axes1.flatten())):
    fig1.delaxes(axes1.flatten()[i])

# Hide unused subplots for fig2 as well
for i in range(len(skip_values), len(axes2.flatten())):
    fig2.delaxes(axes2.flatten()[i])
    
    

fig1.suptitle(f"ResNets with amount layers = {num_hidden + 2} and different weights of shortcut", fontsize=16)
fig2.suptitle(f"Eigenvalue moduli for different shortcut strengths", fontsize=16)

# Save both figures
fig1.tight_layout(rect=[0, 0, 1, 0.96])
fig2.tight_layout(rect=[0, 0, 1, 0.96])

fig1.savefig(subfolder + '/decision_boundaries_skip_params.png', dpi=900, bbox_inches='tight', facecolor='white')
fig2.savefig(subfolder + '/eigenvalue_moduli_skip_params.png', dpi=900, bbox_inches='tight', facecolor='white')


plt.show()

# Singular value computations of Jacobian and plotting
We want to determine singular points in the compact space

In [ ]:
# Define a grid over the input space.
grid_size = 200 # Adjust as needed.

importlib.reload(plots.plots) # Reload the module
from plots.plots import psi_manual, model_to_func, sv_plot
        
# Put the model in evaluation mode.
model = model_base
model.eval()
func = model_to_func(model)  # Add artificial batch dimension which is needed because of batch normalization layer BatchNorm1d and remove it again from the model output.

sv_plot(func, s_index = 0, title = f'Largest SV without output layer')
sv_plot(func, v_index = 1, title = f'Second largest SV of Jacobian without output layer')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Put the model in evaluation mode.
model.eval()

num_hidden = model.num_hidden


# Prepare figure and axes
fig, axes = plt.subplots(2, num_hidden + 2, figsize=(5 * (num_hidden + 2), 10))  # Adjust figsize if needed

for layer in range(num_hidden + 2):
    func = model_to_func(model, from_layer=0, to_layer = layer)

    
    ax = axes[0, layer] if num_hidden > 1 else axes[0]
    cs = sv_plot(func, v_index = 0, ax = ax, grid_size=100)
    # fig.colorbar(cs, ax=ax)
    ax.set_title(f'Min SV\n layer_in = 0, layer_out = {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')
    
    

    # Plot largest singular value (index 0) - second row
    ax = axes[1, layer] if num_hidden > 1 else axes[1]
    cs = sv_plot(func, v_index = 1, ax = ax, grid_size=100)
    # fig.colorbar(cs, ax=ax)
    ax.set_title(f'Max SV\n layer_in = 0, layer_out = {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()


In [ ]:
# Put the model in evaluation mode.
model.eval()

# Prepare figure and axes
fig, axes = plt.subplots(2, num_hidden + 1, figsize=(5 * (num_hidden + 1), 10))  # Adjust figsize if needed

for layer in range(num_hidden + 1):
    func = model_to_func(model, from_layer=layer, to_layer = layer)
    
    ax = axes[0, layer] if num_hidden > 1 else axes[0]
    cs = sv_plot(func, v_index= 0 , ax = ax, grid_size=100)
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Max EV mod\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')
    

    # Plot largest singular value (index 0) - second row
    ax = axes[1, layer] if num_hidden > 1 else axes[1]
    cs = sv_plot(func, v_index=1, ax = ax, grid_size=100)
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Min EV mod\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')

plt.tight_layout()

plt.savefig(subfolder + '/SV_each_layer.png', dpi=600, bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
import plots.plots; importlib.reload(plots.plots)
from plots.plots import sv_plot

# Put the model in evaluation mode.
skip = 0.0
model = models_with_skipparams[skip]
model.eval()

# Prepare figure and axes
fig, axes = plt.subplots(2, num_hidden + 1, figsize=(5 * (num_hidden + 1), 10))  # Adjust figsize if needed

for layer in range(num_hidden + 1):
    func = model_to_func(model, from_layer=layer, to_layer = layer)
    
    ax = axes[0, layer] if num_hidden > 1 else axes[0]
    cs = sv_plot(func, v_index= 0 , ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Max EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')
    

    # Plot largest singular value (index 0) - second row
    ax = axes[1, layer] if num_hidden > 1 else axes[1]
    cs = sv_plot(func, v_index=1, ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Min EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')

plt.tight_layout()

plt.savefig(subfolder + '/EV_each_layer' + str(skip) + '.png', dpi=600, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
import plots.plots; importlib.reload(plots.plots)
from plots.plots import sv_plot

# Put the model in evaluation mode.
skip = 1.0
model = models_with_skipparams[skip]
model.eval()

# Prepare figure and axes
fig, axes = plt.subplots(2, num_hidden + 1, figsize=(5 * (num_hidden + 1), 10))  # Adjust figsize if needed

for layer in range(num_hidden + 1):
    func = model_to_func(model, from_layer=layer, to_layer = layer)
    
    ax = axes[0, layer] if num_hidden > 1 else axes[0]
    cs = sv_plot(func, v_index= 0 , ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Max EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')
    

    # Plot largest singular value (index 0) - second row
    ax = axes[1, layer] if num_hidden > 1 else axes[1]
    cs = sv_plot(func, v_index=1, ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Min EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')

plt.tight_layout()

plt.savefig(subfolder + '/EV_each_layer' + str(skip) + '.png', dpi=600, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
import plots.plots; importlib.reload(plots.plots)
from plots.plots import sv_plot

# Put the model in evaluation mode.
skip = 2.0
model = models_with_skipparams[skip]
model.eval()

# Prepare figure and axes
fig, axes = plt.subplots(2, num_hidden + 1, figsize=(5 * (num_hidden + 1), 10))  # Adjust figsize if needed

for layer in range(num_hidden + 1):
    func = model_to_func(model, from_layer=layer, to_layer = layer)
    
    ax = axes[0, layer] if num_hidden > 1 else axes[0]
    cs = sv_plot(func, v_index= 0 , ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Max EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')
    

    # Plot largest singular value (index 0) - second row
    ax = axes[1, layer] if num_hidden > 1 else axes[1]
    cs = sv_plot(func, v_index=1, ax = ax, grid_size=100, output_type='eigmods')
    fig.colorbar(cs, ax=ax)
    ax.set_title(f'Min EV\n layer {layer}')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_aspect('equal')

plt.tight_layout()

plt.savefig(subfolder + '/EV_each_layer' + str(skip) + '.png', dpi=600, bbox_inches='tight', facecolor='white')
plt.show()

# XOR data set

In [ ]:
from models.training import make_xor

train_loader, test_loader = make_xor(1)

In [ ]:
# Model Params
num_hidden = 6 # number of hidden layers. The total network has additionl 2 layers: input to hidden and hidden to output
input_dim = 2
hidden_dim = 2
output_dim = 1
activation = 'tanh' #'relu' and 'tanh' are supported

# Training Params
load_file = None
cross_entropy = True #True supported with binary classification only
num_epochs = 300



In [ ]:
footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'

model_base, acc_base, losses_base = train_until_threshold(ResNet,
    train_loader, test_loader,
    load_file = load_file, max_retries=5, threshold=0.95,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=0, activation=activation
)

plot_loss_curve(losses_base, title=f"Base Model Loss Curve", filename = subfolder + '/xor_ff6hidden')

In [ ]:
X_test, y_test = next(iter(test_loader))
plot_decision_boundary(model_base, X_test, y_test, show=True, file_name= 'ff6hiddenxor' + str(num_hidden), footnote = footnote, amount_levels= 100)
plot_level_sets(model_base, show=True, file_name= subfolder + '/ff6hiddenxor_contour' + str(num_hidden), footnote = footnote, amount_levels= 20)

In [ ]:
hidden_dim = 3
num_hidden = 2

footnote = f'num_hidden={num_hidden}, hidden_dim={hidden_dim}, output_dim={output_dim}, act={activation}, seed={seed}, ce={cross_entropy}'

model_wide, acc_wide, losses_wide = train_until_threshold(ResNet,
    train_loader, test_loader,
    load_file = load_file, max_retries=5, threshold=0.95,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=0, activation=activation
)

plot_loss_curve(losses_wide, title=f"Wide Model Loss Curve", filename = subfolder + '/ffxorloss')

In [ ]:
X_test, y_test = next(iter(test_loader))
plot_decision_boundary(model_wide, X_test, y_test, show=True, file_name= 'ffxor' + str(num_hidden), footnote = footnote, amount_levels= 100)
plot_level_sets(model_wide, show=True, file_name= subfolder + '/ffxor_contour' + str(num_hidden), footnote = footnote, amount_levels= 30)